In [1]:
import os
from lxml import etree 

doc = etree.parse('../Data/xml_data/provenance.xml')
provenance = doc.findall('recordList/record/provenance/provenance.project.txt')
for item in provenance:
    if item.text == None:
        provenance.remove(item)
    else:
        continue

In [2]:
auction_items = list()
for elements in provenance:
    element = elements.text
    element_list = element.split(';')
    for elem in element_list:
        if 'Sale' in elem:
            auction_items.append(elem)
        elif 'sale' in elem:
            auction_items.append(elem)
        else:
            continue

In [4]:
for item in auction_items[0:101]:
    test_item = item
    start_source = ''
    sale_section = ''
    owner = ''
    BDDate = ''
    place_collection = ''
    place_auction = ''
    auctionhouse = ''
    date_auction = ''
    kavel_no = ''
    attribution = ''
    description = ''
    pendant = ''
    pendant_extra = ''
    price = ''
    buyer = ''
    buyer_city = ''
    supporter = ''
    source = ''
    anonymous = False
    his = False
    her = False
    start_item = item

    certain_uncertain = item.split('(')
    if '?' in certain_uncertain[0]:
        #provenance data is uncertain
        proper_format = '? '
        readable_sent = 'Possibly sold by'
    else:
        #provenance data is certain
        proper_format = ''
        readable_sent = 'Sold by'
    #start_source
    source_split = item.split('}')
    if len(source_split) >1:
        if ('sale' in source_split[0] or 'Sale' in source_split[0]) and not 'sale' in source_split[1] and not 'Sale' in source_split[1]:
            Print('Wrongly assigned due to source information')
            continue
        if 'sale' in source_split[1]:
            start_source = source_split[0].replace('{', '').strip()
    #source
    if start_source == '':
        source_split = item.split('{')
        if len(source_split) >1:
            source = source_split[1].replace('}', '').strip()

    #item without source
    if start_source != '':
        item = item.replace(start_source, '').strip()
        start_source = '{'+start_source+'}'
    if source != '':
        item = item.replace(source, '').strip()
        source = '{'+source+'}'

    comma_split = item.split(',')
    #name of owner
    if 'anonymous sale' in comma_split[0]:
        anonymous = True
        owner = ''
        if 'section' in comma_split[0]:
            square_split = item.split('[')
            sale_section = square_split[1]
            sale_section = sale_section.split(']')
            sale_section = sale_section[0].strip()
    elif 'his' in comma_split[0] and 'sale' in comma_split[0]:
        his = True
        owner = ''
    elif 'her' in comma_split[0] and 'sale' in comma_split[0]:
        her = True
        owner = ''
    else:
        owner = comma_split[1].split('(')
        owner = owner[0].strip()

    #if et al. in owner
    if 'et al.' in owner:
        square_split = item.split('[')
        if len(square_split) >1:
            if 'section' in square_split[1]:
                square_split = square_split[1].split(']')
                owner = owner + ' [' + square_split[0].strip() + ']'
                item = item.replace(square_split[0],'')
    #new comma split in case there was a section item
    comma_split = item.split(',')
    if 'et al.' in comma_split[2]:
        owner = owner + ', ' + comma_split[2]
        owner = owner.strip()

    #birth/death date owner and place of collection before sale
    if owner != '':
        BDDate_item = comma_split[1]
        if '†' in BDDate_item:
            BDdate = '†'
        elif ('1' in BDDate_item or '2' in BDDate_item) and not 'collection' in comma_split[2]:
            BDsplit = BDDate_item.split('(')
            BDDate = BDsplit[1].replace(')', '').strip()
            if not BDsplit[1].endswith(')'):
                place_collection = comma_split[2].replace(')','').strip()                

    #place of auction and auction house
    #for anonymous sale:
    if owner == '':
        place_auction_item = comma_split[1]
        auction_split = place_auction_item.split('(')
        place_auction = auction_split[0].strip()
        if len(auction_split) >1:
            auctionhouse = auction_split[1].replace(')', '').strip()
    #for known owner

    if owner != '' and place_collection == '':
        if 'et al.' in owner and ',' in owner:
            place_auction_item = comma_split[3]
        else:
            place_auction_item = comma_split[2]
        auction_split = place_auction_item.split('(')
        place_auction = auction_split[0].strip()
        if len(auction_split) > 1:
            auctionhouse = auction_split[1].replace(')', '').strip()
    if owner != '' and place_collection != '':
        if 'et al.' in owner and ',' in owner:
            place_auction_item = comma_split[4]
        else:
            place_auction_item = comma_split[3]
        auction_split = place_auction_item.split('(')
        place_auction = auction_split[0].strip()
        if len(auction_split) > 1:
            auctionhouse = auction_split[1].replace(')', '').strip()
            if '1' in auctionhouse:
                date_auction = auctionhouse
                auctionhouse = ''

    if '1' in auctionhouse:
        date_auction = auctionhouse
        auctionhouse = ''
    #date auction
    #for anonymous sale
    if owner == '':
        if 'no.' not in comma_split[2]:
            date_auction = comma_split[2].strip()
    #for known owner
    elif owner != '' and place_collection == '' and date_auction =='':
        if 'et al.' in owner and ',' in owner:
            if 'no.' not in comma_split[4]:
                date_auction = comma_split[4].strip()
        else:
            if 'no.' not in comma_split[3]:
                date_auction = comma_split[3].strip()
    elif owner != '' and place_collection != '' and len(comma_split) >4 and date_auction =='':
        if 'et al.' in owner and ',' in owner:
            if 'no.' not in comma_split[5]:
                date_auction = comma_split[5].strip()
        else:
            if 'no.' not in comma_split[4]:
                date_auction = comma_split[4].strip()

    #kavel no, price, attribution, pendant, pendant extra
    for item in comma_split:
        if 'no.' in item and kavel_no == '':
            kavel_no = item.strip()
        money = (' fl. ', ' frs. ', ' bf ', '£', ' gns ', ' s ', '$', ' DM ', ' CHF ', ' CAD ', ' ATS ')
        for valuta in money:
            if valuta in item:
                price = item
                price_index = comma_split.index(price)
                if len(comma_split) > price_index+2:
                    if comma_split[price_index+1].isnumeric():
                        price = price+','+comma_split[price_index+1]
                        price = price.strip()
        if ' to ' in item:
            buyer_index = item
            buyer = item.strip()
            #is the city of the buyer specified
            buyer_index = comma_split.index(buyer_index)
            if (len(comma_split) > (buyer_index + 1)) and 'with support' not in comma_split[(buyer_index+1)]:
                buyer_city = comma_split[(buyer_index+1)].strip()
        if 'with support' in item:
            supporter = item.strip()
        if 'without attribution' in item:
            attribution = item.strip()
        if ' as ' in item:
            attribution = item.strip()
        if 'with pendant' in item:
            pendant = item.strip()
        if pendant != '':                     
            if 'no.' in item:
                item_index_1 = comma_split.index(item)
                for item in comma_split:
                    for valuta in money:
                        if valuta in item:
                            item_index_2 = comma_split.index(item)
                            pendant_extra = comma_split[int(item_index_1):item_index_2]
                            pendant_extra = ','.join(pendant_extra)

    para_split = item.split('(')
    for item in para_split:
        if item.startswith("'"):
            description = item.split(')')
            description = description[0].strip()

    #proper format and readable sentence
    #proper format
    if start_source != '':
        proper_format = f"{start_source} {proper_format}"
    if anonymous == True and sale_section == '':
        proper_format = f"{proper_format}anonymous sale"
    if anonymous == True and sale_section != '':
        proper_format = f"{proper_format}anonymous sale [{sale_section}]"
    if his == True:
        proper_format = f"{proper_format}his sale"
    if her == True:
        proper_format = f"{proper_format}her sale"
    if anonymous == False and his == False and her == False:
        proper_format = f"{proper_format}sale"
    if owner != '' and anonymous == False and his == False and her == False:
        proper_format = f"{proper_format}, {owner}"
    if BDDate != '' and place_collection == '':
        proper_format = f"{proper_format} ({BDDate})"
    if BDDate != '' and place_collection != '':
        proper_format = f"{proper_format} ({BDDate}, {place_collection})"
    if place_auction != '':
        proper_format = f"{proper_format}, {place_auction}"
    if auctionhouse != '':
        proper_format = f"{proper_format} ({auctionhouse})"
    if date_auction != '':
        proper_format = f"{proper_format}, {date_auction}"
    if kavel_no != '':
        proper_format = f"{proper_format}, {kavel_no}"
    if attribution != '':
        proper_format = f"{proper_format}, {attribution}"
    if description != '':
        proper_format = f"{proper_format} ({description})"
    if pendant != '':
        proper_format = f"{proper_format}, {pendant}"
    if pendant_extra != '':
        proper_format = f"{proper_format}, {pendant_extra}"
    if price != '':
        proper_format = f"{proper_format}, {price}"
    if buyer != '':
        proper_format = f"{proper_format}, {buyer}"
    if buyer_city != '':
        proper_format = f"{proper_format}, {buyer_city}"
    if supporter != '':
        proper_format = f"{proper_format}, {supporter}"
    if source != '':
        proper_format = f"{proper_format} {source}"

    #readable sentence
    if anonymous == True and sale_section == '':
        readable_sent = f"{readable_sent} anonymous"
    if anonymous == True and sale_section != '':
        readable_sent = f"{readable_sent} anonymous [{sale_section}]"
    if his == True:
        readable_sent = f"{readable_sent} him"
    if her == True:
        readable_sent = f"{readable_sent} her"
    if anonymous == False and his == False and her == False and owner != '':
        readable_sent = f"{readable_sent} {owner}"
    if BDDate != '' and place_collection == '':
        readable_sent = f"{readable_sent} ({BDDate})"
    if BDDate != '' and place_collection != '':
        readable_sent = f"{readable_sent} ({BDDate}, item previously kept in {place_collection})"
    if place_auction != '':
        readable_sent = f"{readable_sent}, auctioned off in {place_auction}"
    if auctionhouse != '':
        readable_sent = f"{readable_sent} (auction house {auctionhouse})"
    if date_auction != '':
        readable_sent = f"{readable_sent} on {date_auction}"
    if kavel_no != '':
        readable_sent = f"{readable_sent}, {kavel_no}"
    if attribution != '':
        readable_sent = f"{readable_sent}, {attribution}"
    if description != '':
        readable_sent = f"{readable_sent} ({description})"
    if pendant != '':
        readable_sent = f"{readable_sent}, {pendant}"
    if pendant_extra != '':
        readable_sent = f"{readable_sent}, {pendant_extra}"
    if price != '':
        readable_sent = f"{readable_sent}, bought for {price}"
    if buyer != '':
        readable_sent = f"{readable_sent}, sold {buyer}"
    if buyer_city != '':
        readable_sent = f"{readable_sent}, {buyer_city}"
    if supporter != '':
        readable_sent = f"{readable_sent}, {supporter}"
    readable_sent = f"{readable_sent}."

    #test where mistakes are made
    if proper_format != test_item.strip():
        print('Old format: ',test_item)
        print('New format: ',proper_format, '\n')

 sale, A.W.M. Mensing(1866-1936, Amsterdam), Amsterdam (F. Muller), 17 November 1936 sqq., no. 9A
Proper format: sale, A.W.M. Mensing (1866-1936, Amsterdam), Amsterdam (F. Muller), 17 November 1936 sqq., no. 9A


{Coll. cat. Weill III, p. 463.} sale, Irwin Laughlin et al. [section 'The property of a gentleman', probably David Weill], London (Sotheby's), 10 June 1959, no. 131, £ 1,100, to the dealer Colnaghi for the museum, with support from the Vereniging Rembrandt
Proper format: {Coll. cat. Weill III, p. 463.} sale, Irwin Laughlin et al. [section 'The property of a gentleman' [section 'The property of a gentleman', probably David Weill], London (Sotheby's), 10 June 1959, no. 131, £ 1,100, to the dealer Colnaghi for the museum, with support from the Vereniging Rembrandt


 sale, Sächsischen Staatssammlungen Johanneum, Grünes Gewölbe, Gemäldegalerie in Dresden, Dresden (Rudolph Lepke), 12 (13) October 1920 sqq., no. 780, DM 250,000, to the dealer Heilbronner, Berlin
Proper format: sale,